## RとTidyデータの原則を使ったK-Meansクラスタリングの探求

### [**講義前クイズ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

このレッスンでは、TidymodelsパッケージやRエコシステム内の他のパッケージ（友達と呼びましょう 🧑‍🤝‍🧑）を使ってクラスタを作成する方法と、以前インポートしたナイジェリア音楽データセットを使用する方法を学びます。K-Meansクラスタリングの基本について説明します。前のレッスンで学んだように、クラスタを扱う方法は多岐にわたり、使用する方法はデータに依存します。ここでは、最も一般的なクラスタリング手法であるK-Meansを試してみましょう。それでは始めましょう！

このレッスンで学ぶ用語:

- シルエットスコアリング
- エルボー法
- 慣性（Inertia）
- 分散（Variance）

### **イントロダクション**

[K-Meansクラスタリング](https://wikipedia.org/wiki/K-means_clustering)は、信号処理の分野から派生した手法です。この手法は、データの特徴の類似性に基づいて、データを`k個のクラスタ`に分割・区分するために使用されます。

クラスタは、[ボロノイ図](https://wikipedia.org/wiki/Voronoi_diagram)として視覚化することができ、これは点（または「シード」）とその対応する領域を含みます。

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Jen Looperによるインフォグラフィック</figcaption>

K-Meansクラスタリングの手順は以下の通りです:

1. データサイエンティストが作成したいクラスタの数を指定します。

2. 次に、アルゴリズムがデータセットからランダムにK個の観測値を選択し、それをクラスタの初期中心（すなわちセントロイド）として使用します。

3. 次に、残りの各観測値を最も近いセントロイドに割り当てます。

4. 次に、各クラスタの新しい平均を計算し、セントロイドをその平均に移動させます。

5. セントロイドが再計算されたので、すべての観測値を再度確認し、別のクラスタに近いかどうかを確認します。すべてのオブジェクトは、更新されたクラスタ平均を使用して再割り当てされます。このクラスタ割り当てとセントロイド更新のステップは、クラスタ割り当てが変化しなくなるまで（すなわち、収束が達成されるまで）繰り返されます。通常、各新しい反復でセントロイドの移動がわずかになり、クラスタが静的になるとアルゴリズムは終了します。

<div>

> 初期のk個の観測値をランダムに選択するため、手順を適用するたびに結果が若干異なる可能性があることに注意してください。このため、ほとんどのアルゴリズムでは複数の*ランダムスタート*を使用し、最も低いWCSSを持つ反復を選択します。そのため、*望ましくない局所最適解*を避けるために、常に複数の*nstart*値でK-Meansを実行することを強くお勧めします。

</div>

この短いアニメーションは、Allison Horstの[アートワーク](https://github.com/allisonhorst/stats-illustrations)を使用してクラスタリングプロセスを説明しています:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>@allison_horstによるアートワーク</figcaption>

クラスタリングにおいて基本的な疑問が生じます。それは、データをいくつのクラスタに分けるべきかということです。K-Meansの欠点の1つは、`k`、つまり`セントロイド`の数を事前に決める必要がある点です。幸いなことに、`エルボー法`を使えば、`k`の良い初期値を推定するのに役立ちます。これをすぐに試してみましょう。

### 

**前提条件**

[前回のレッスン](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb)でデータセットを分析し、多くの可視化を行い、興味のある観測値にデータセットをフィルタリングしました。このレッスンをぜひ確認してください！

このモジュールを進めるにはいくつかのパッケージが必要です。以下のコマンドでインストールできます:  
`install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

または、以下のスクリプトを使用すると、このモジュールを完了するために必要なパッケージがインストールされているか確認し、不足している場合は自動的にインストールします。


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


さあ、始めましょう！

## 1. データとのダンス: 最も人気のある音楽ジャンルを3つに絞る

これは前回のレッスンで行った内容の復習です。データを切り分けて分析してみましょう！


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 これはうまくいきましたね！

## 2. データをさらに探索する

このデータはどれくらいクリーンでしょうか？ボックスプロットを使って外れ値を確認してみましょう。外れ値が少ない数値列に注目します（もちろん、外れ値を除去することも可能です）。ボックスプロットはデータの範囲を示し、どの列を使用するか選ぶのに役立ちます。ただし、ボックスプロットは分散を示さないため、クラスタリングに適したデータを評価する上で重要な要素である分散については別途考慮が必要です。詳しくは[この議論](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot)をご覧ください。

[ボックスプロット](https://en.wikipedia.org/wiki/Box_plot)は、`数値`データの分布を視覚的に表現するために使用されます。それでは、人気のある音楽ジャンルとともに、すべての数値列を*選択*することから始めましょう。


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


選択ヘルパー`where`がどれだけ簡単にしてくれるか分かりますか💁？他の便利な関数については[こちら](https://tidyselect.r-lib.org/)をチェックしてください。

数値特徴ごとにボックスプロットを作成する予定ですが、ループを使うのは避けたいので、データを*縦長*の形式に再整形しましょう。これにより、`facets`（データの各サブセットを表示するサブプロット）を活用できるようになります。


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


もっと長くなります！さあ、`ggplot`を使う時間です！では、どの`geom`を使いましょうか？


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


簡単だね！

さて、このデータは少しノイズがあることがわかります。各列をボックスプロットで観察すると、外れ値が見られます。この外れ値をデータセットから取り除くこともできますが、それではデータがかなり少なくなってしまいます。

とりあえず、クラスタリングの演習で使用する列を選びましょう。範囲が似ている数値列を選びます。`artist_top_genre`を数値にエンコードすることもできますが、今回はそれを除外します。


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Rでk-meansクラスタリングを計算する

Rでは、組み込みの`kmeans`関数を使用してk-meansを計算できます。詳細は`help("kmeans()")`を参照してください。`kmeans()`関数は、すべての列が数値型であるデータフレームを主な引数として受け取ります。

k-meansクラスタリングを使用する際の最初のステップは、最終的なソリューションで生成されるクラスタ数（k）を指定することです。データセットから抽出した3つの音楽ジャンルがあることを知っているので、3を試してみましょう:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


kmeansオブジェクトには、`help("kmeans()")`で詳しく説明されているいくつかの情報が含まれています。ここでは、その中のいくつかに焦点を当ててみましょう。このデータは、サイズがそれぞれ65、110、111の3つのクラスターにグループ化されていることがわかります。また、出力には、5つの変数にわたる3つのグループのクラスター中心（平均値）も含まれています。

クラスタリングベクトルは、各観測値のクラスター割り当てを示しています。`augment`関数を使用して、元のデータセットにクラスター割り当てを追加してみましょう。


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


完璧です！これでデータセットを3つのグループに分割しました。それでは、クラスタリングの結果はどれくらい良いのでしょうか 🤷？`Silhouetteスコア`を見てみましょう。

### **Silhouetteスコア**

[Silhouette分析](https://en.wikipedia.org/wiki/Silhouette_(clustering))は、得られたクラスタ間の分離距離を調べるために使用できます。このスコアは-1から1の範囲で変動し、スコアが1に近い場合、クラスタは密集しており、他のクラスタと明確に分離されています。一方、スコアが0に近い場合、クラスタは重なり合っており、サンプルが隣接するクラスタの境界付近に非常に近いことを示します。[参考](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam)。

平均Silhouette法では、異なる*k*の値に対して観測値の平均Silhouetteを計算します。平均Silhouetteスコアが高いほど、良好なクラスタリングを示します。

クラスタパッケージの`silhouette`関数を使用して、平均Silhouette幅を計算します。

> Silhouetteは、[ユークリッド距離](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance")や[マンハッタン距離](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance")など、任意の[距離](https://en.wikipedia.org/wiki/Distance "Distance")メトリックで計算することができます。これらの距離については[前のレッスン](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb)で説明しました。


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


私たちのスコアは **.549** で、ちょうど中間に位置しています。これは、私たちのデータがこのタイプのクラスタリングに特に適しているわけではないことを示しています。この推測を視覚的に確認できるかどうか見てみましょう。[factoextra パッケージ](https://rpkgs.datanovia.com/factoextra/index.html)は、クラスタリングを視覚化するための関数（`fviz_cluster()`）を提供しています。


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


クラスターの重なりは、今回のデータがこのタイプのクラスタリングに特に適していないことを示していますが、続けてみましょう。

## 4. 最適なクラスター数の決定

K-Meansクラスタリングでよく出てくる基本的な疑問の一つは、「既知のクラスラベルがない場合、データをいくつのクラスターに分ければよいのか？」という点です。

これを調べる一つの方法として、データサンプルを使って`クラスター数を増やしながら一連のクラスタリングモデルを作成`（例: 1～10まで）し、**シルエットスコア**のようなクラスタリング指標を評価する方法があります。

最適なクラスター数を決定するために、異なる*k*の値に対してクラスタリングアルゴリズムを計算し、**クラスター内平方和（WCSS: Within Cluster Sum of Squares）**を評価してみましょう。クラスター内平方和（WCSS）はクラスタリングのコンパクトさを測る指標で、値が小さいほどデータポイントが近いことを意味します。そのため、WCSSはできるだけ小さい方が望ましいです。

それでは、1から10までの異なる`k`の選択がこのクラスタリングに与える影響を調べてみましょう。


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


各クラスタリングアルゴリズムにおける中心 *k* の総クラスタ内平方和 (tot.withinss) を求めたので、[肘法](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) を使用して最適なクラスタ数を見つけます。この方法では、クラスタ数の関数としてWCSSをプロットし、[曲線の肘](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") をクラスタ数として選択します。


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


プロットは、クラスタ数が1から2に増加するにつれてWCSSが大幅に減少（つまり、*密集度*が向上）し、さらに2から3クラスタへの増加でも顕著な減少を示しています。その後、減少はそれほど顕著ではなくなり、約3クラスタのところでチャートに`肘` 💪が現れます。これは、データポイントが2～3の適度に分離されたクラスタに分かれている良い指標です。

ここで、`k = 3`のクラスタリングモデルを抽出する準備が整いました：

> `pull()`: 単一の列を抽出するために使用
>
> `pluck()`: リストなどのデータ構造をインデックスするために使用


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


素晴らしいですね！取得したクラスターを視覚化してみましょう。`plotly`を使ってインタラクティブにしてみませんか？


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


おそらく、各クラスタ（異なる色で表される）がそれぞれ異なるジャンル（異なる形で表される）を持つことを期待していたかもしれません。

モデルの精度を確認してみましょう。


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


このモデルの精度は悪くはありませんが、特別良いとも言えません。このデータは、K-Meansクラスタリングに適していない可能性があります。データが不均衡で、相関が少なく、列の値の間に大きな分散があるため、うまくクラスタリングできないのです。実際、形成されるクラスタは、上で定義した3つのジャンルカテゴリに大きく影響されている、または偏っている可能性が高いです。

それでも、これは非常に学びの多いプロセスでした！

Scikit-learnのドキュメントでは、このようにクラスタがあまり明確に区別されていないモデルは「分散」の問題を抱えていると説明されています。

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Scikit-learnのインフォグラフィック</figcaption>



## **分散**

分散は、「平均からの二乗差の平均」と定義されます [出典](https://www.mathsisfun.com/data/standard-deviation.html)。このクラスタリング問題の文脈では、データセットの数値が平均から少し離れすぎる傾向があることを指します。

✅ ここで、この問題を解決するためのさまざまな方法を考える良い機会です。データをもう少し調整しますか？別の列を使用しますか？別のアルゴリズムを試しますか？ヒント: データを正規化するために[スケーリング](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/)を試し、他の列をテストしてみてください。

> この '[分散計算機](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' を使って、概念をもう少し理解してみてください。

------------------------------------------------------------------------

## **🚀チャレンジ**

このノートブックで時間をかけてパラメータを調整してみてください。データをさらにクリーンアップする（例えば外れ値を削除する）ことで、モデルの精度を向上させることができますか？特定のデータサンプルに重みを付けることもできます。他にどのような方法でより良いクラスタを作成できますか？

ヒント: データをスケーリングしてみてください。ノートブックには、データ列を範囲的により似たものにするための標準スケーリングを追加するコメント付きコードがあります。シルエットスコアは下がりますが、エルボーグラフの「折れ目」が滑らかになります。これは、データをスケーリングしないままにすると、分散の少ないデータがより大きな重みを持つようになるためです。この問題についてもう少し読みたい場合は、[こちら](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226)をご覧ください。

## [**講義後のクイズ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **復習と自己学習**

-   K-Meansシミュレーター [こちら](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/) を見てみてください。このツールを使ってサンプルデータポイントを視覚化し、そのセントロイドを決定できます。データのランダム性、クラスタ数、セントロイド数を編集できます。これにより、データがどのようにグループ化できるかのアイデアが得られますか？

-   また、スタンフォード大学の [K-Meansに関するハンドアウト](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) も見てみてください。

K-Meansクラスタリングに適したデータセットで新たに習得したクラスタリングスキルを試してみたいですか？以下をご覧ください：

-   [クラスタリングモデルのトレーニングと評価](https://rpubs.com/eR_ic/clustering)（Tidymodelsとその関連ツールを使用）

-   [K-meansクラスタ分析](https://uc-r.github.io/kmeans_clustering)、UC Business Analytics R Programming Guide

- [tidyデータの原則を用いたK-meansクラスタリング](https://www.tidymodels.org/learn/statistics/k-means/)

## **課題**

[異なるクラスタリング手法を試してみる](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## 感謝の言葉：

[Jen Looper](https://www.twitter.com/jenlooper) さん、Python版のオリジナルモジュールを作成していただきありがとうございます ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) さん、Rをより親しみやすく魅力的にする素晴らしいイラストを作成していただきありがとうございます。彼女の[ギャラリー](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM)でさらに多くのイラストをご覧ください。

楽しい学びを！

[Eric](https://twitter.com/ericntay)、Gold Microsoft Learn Student Ambassador

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>@allison_horstによるアートワーク</figcaption>



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確さが含まれる可能性があります。元の言語で記載された原文が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤認について、当社は一切の責任を負いません。
